# 1. Merge all files of nhaphonet.vn

In [2]:
import numpy as np
import pandas as pd
import re
import os

In [3]:
folder_path = "D:\\OneDrive\\KiotViet\\Python_for_work\\KFinance\\CSV_crawl_data"
file_list = os.listdir(folder_path)

merged_df = pd.DataFrame()

# Loop through each file in the folder
for file in file_list:
    if "nhaphonet_" in file and file.endswith(".csv"):
        file_path = os.path.join(folder_path, file)

        # Specify the encoding as 'utf-8' or the appropriate encoding for your files
        try:
            df = pd.read_csv(file_path, encoding='utf-8')
            merged_df = pd.concat([merged_df, df], ignore_index=True)
        except UnicodeDecodeError:
            print(f"Error reading file: {file_path}. Skipping.")

print(len(merged_df))

4080


In [4]:
merged_df = merged_df.dropna()
merged_df.to_csv(r"D:\OneDrive\KiotViet\Python_for_work\KFinance\CSV_crawl_data\all_bds_nhaphonet.csv", index=False, mode = 'w', encoding="utf-8-sig")

# 2. Clean data 

In [5]:
df = pd.read_csv(r"D:\OneDrive\KiotViet\Python_for_work\KFinance\CSV_crawl_data\all_bds_nhaphonet.csv")
df.head()
print(f"The total rows of dataframe is {len(df)}")

The total rows of dataframe is 2040


In [6]:
df = df[~df['Price'].str.contains('Thỏa thuận')]
df = df.drop_duplicates()
print(f"The total rows of dataframe is {len(df)}")

The total rows of dataframe is 1956


2.0 Xử lý diện tích

In [7]:
df['Area'].head()

0    100m2
1     60m2
2     90m2
3     24m2
4     73m2
Name: Area, dtype: object

In [8]:
df['area_value'] = df['Area'].apply(lambda x: float(x.replace('m2', '')))

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1956 entries, 0 to 2039
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Title       1956 non-null   object 
 1   Price       1956 non-null   object 
 2   Area        1956 non-null   object 
 3   Location    1956 non-null   object 
 4   type        1956 non-null   object 
 5   area_value  1956 non-null   float64
dtypes: float64(1), object(5)
memory usage: 107.0+ KB


2.1 Xử lý price

In [10]:
#Tạo cột 'price_per_m2'

# df['price_per_m2'] = df['Price'].apply(lambda x: str(x.replace('Giá\n:\n', '')))

def convert_price_string_to_float(price_string):
    return float(price_string.replace(',', '.'))

price_pattern = re.compile(r'([\d,]+) (triệu /tháng)')
price_pattern_m2 = re.compile(r'([\d,]+) (triệu /m2)')
price_pattern_ty = re.compile(r'([\d,]+) (tỷ /tháng)')
price_pattern_ty_m2 = re.compile(r'([\d,]+) (tỷ /m2)')
price_pattern_trieu = re.compile(r'([\d,]+) (triệu)')

#1.Nếu Price ghi X triệu/tháng và Area ghi Y m2 thì => Cột 'price_per_m2' thành X*1000000/Y
df['price_value_trieu'] = df['Price'].apply(lambda x: convert_price_string_to_float(price_pattern.search(x).group(1)) if price_pattern.search(x) is not None else None)
df['price_per_m2_trieu'] = 1000000 * df['price_value_trieu'] / df['area_value']

#2. Nếu Price ghi X triệu/m2 thì cột 'price_per_m2' là X*1000000
df['price_value_trieu2'] = df['Price'].apply(lambda x: convert_price_string_to_float(price_pattern_m2.search(x).group(1)) if price_pattern_m2.search(x) is not None else None)
df['price_per_m2_trieu2'] = 1000000 * df['price_value_trieu2']

#3.Nếu Price ghi X tỷ/tháng và Area ghi Y m2 thì => Cột 'price_per_m2' thành X*1000000000/Y
df['price_value_ty'] = df['Price'].apply(lambda x: convert_price_string_to_float(price_pattern_ty.search(x).group(1)) if price_pattern_ty.search(x) is not None else None)
df['price_per_m2_ty'] = 1000000000 * df['price_value_ty'] / df['area_value']

#4.Nếu Price ghi X tỷ/m2 và Area ghi Y m2 thì => Cột 'price_per_m2' thành X*1000000000
df['price_value_ty2'] = df['Price'].apply(lambda x: convert_price_string_to_float(price_pattern_ty_m2.search(x).group(1)) if price_pattern_ty_m2.search(x) is not None else None)
df['price_per_m2_ty2'] = 1000000000 * df['price_value_ty2']


#3.Nếu Price ghi X triệu và Area ghi Y m2 thì => Cột 'price_per_m2' thành X*1000000/Y
df['price_value_trieu3'] = df['Price'].apply(lambda x: convert_price_string_to_float(price_pattern_trieu.search(x).group(1)) if price_pattern_trieu.search(x) is not None else None)
df['price_per_m2_trieu3'] = 1000000 * df['price_value_trieu3'] / df['area_value']



In [11]:
#4. Từ các cột "price_per_m2_trieu", "price_per_m2_trieu2", "price_per_m2_ty", 'price_per_m2_ty2', 'price_per_m2_trieu3' ta tìm được price_per_m2_final

def calculate_price_per_m2_final(row):
    if not pd.isna(row['price_per_m2_trieu']):
        return row['price_per_m2_trieu']
    elif not pd.isna(row['price_per_m2_trieu2']):
        return row['price_per_m2_trieu2']
    elif not pd.isna(row['price_per_m2_ty']):
        return row['price_per_m2_ty']
    elif not pd.isna(row['price_per_m2_trieu3']):
        return row['price_per_m2_trieu3']
    elif not pd.isna(row['price_per_m2_ty2']):
        return row['price_per_m2_ty2']
    else:
        return np.nan

df['price_per_m2'] = df.apply(calculate_price_per_m2_final, axis=1)

2.2. Xử lý địa chỉ

In [12]:
df['address'] = df['Location']


In [13]:
# Áp dụng hàm tùy chỉnh để tạo cột mới
df = df[['Title','Price','Area','address','area_value','price_per_m2']]
# df = df.drop(columns=['price_per_m2_nghin','price_per_m2_trieu2','price_per_m2_trieu','price_value_nghin','price_value_trieu2']).dropna()
df

,Title,Price,Area,address,area_value,price_per_m2
0,CHO THUÊ NHÀ GÓI 2 MỸ GIA VĨNH THÁI,15 triệu /tháng,100m2,"Nha Trang, Khánh Hòa",100.0,1.500000e+05
1,"--Cần cho thuê nhà gói 2 , khu đô thị MỸ Gia -...",13.5 triệu /tháng,60m2,"lô 3e, Đường Lê Hồng Phong, Ngô Quyền, Hải Phòng",60.0,8.333333e+04
2,"cho thuê nhà ở lô 3e Lê Hồng Phong phù hợp ở, ...",20 triệu /tháng,90m2,"Đường Hoàng Thế Thiện, Hải An, Hải Phòng",90.0,2.222222e+05
3,cho thuê nhà ở lô 3e Lê Hồng Phong- 60...,20 triệu /tháng,24m2,"Lê Chân, Hải Phòng",24.0,8.333333e+05
4,"CHO THUÊ NHÀ MẶT ĐƯỜNG HOÀNG THẾ THIỆN 4 TẦNG,...",12 triệu /tháng,73m2,"Đường Lương Thế Vinh, Thanh Xuân, Hà Nội",73.0,1.643836e+05
...,...,...,...,...,...,...
2035,"+ Diện tích : 240m2 x4 tâng, kinh doanh sầm uấ...",294 triệu /tháng,330m2,"Đường Ngô Thì Nhậm, Hai Bà Trưng, Hà Nội",330.0,8.909091e+05
2036,Cho thuê MBKD nhà mặt phố CAO THẮNG-mặt tiền 9m.,37 triệu /tháng,35m2,"Đường Hàng Bông, Hoàn Kiếm, Hà Nội",35.0,1.057143e+06
2037,"+ Diện tích : 80m2 x 2 tầng, khu đông dân cư. ...",100 triệu /tháng,738m2,"Đường Cửa Nam, Hoàn Kiếm, Hà Nội",738.0,1.355014e+05
2038,Cho thuê MBKD nhà mặt phố NHÀ CHUNG-mặt tiền 5m.,90 triệu /tháng,290m2,"Đường Hàng Mành, Hoàn Kiếm, Hà Nội",290.0,3.103448e+05


In [14]:
df = df.dropna(subset=['price_per_m2'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1955 entries, 0 to 2039
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Title         1955 non-null   object 
 1   Price         1955 non-null   object 
 2   Area          1955 non-null   object 
 3   address       1955 non-null   object 
 4   area_value    1955 non-null   float64
 5   price_per_m2  1955 non-null   float64
dtypes: float64(2), object(4)
memory usage: 106.9+ KB


In [15]:
list_api_keys = [
"N5qb2q5dSfly3fnqWMsDPqVMWUd1yRbTBxbT34hvw_g",
"SsQNhtNfemZJr_FzCYZlub29VeTTjymT3Qm6syC50W8",
"br4m9W3KMhrPa51Vru5M1mWJYCfaVFvWiP3kB2gEQUA",
"spdVYGb-b6cgokZBOuwGG4b3WbmgsxQqv7tudjdOfLc",
"0zeR-GEwo5Od-poIZYTGrw-I80fC2C0b0q4F1Boa8oc",
"WCpDyLiAACoL9zwSFQQa_HboP-HdWXSM601W1N02aeg",
"faJrm5egL12NmLivrDsbqz44I47DhbJQUIg89L3TdIo",
"cdXQHZ3zzsZv-n04kjimcml7e3BRKcR2H3wl8XPi6SU",
"RBRLSkMgpwy6QF0jKsXs-jXEliqtAwwD_jd0DbHrEN8",
"Dd-n66bTKOIKeQ0H6nRvIOMWatnJ_viPxKMNGpxsFa4",
"q-Ao_RRfZNyfQPp2XsnYGbrOItF6SlXN0jObPIKVnk",
"ufLjrWIrvjyLHzL1TZIEOZjKpgolDMBUPXUwoIblgs0",
"PgxoArILmRDFQreVp3pC6bexYfcKIVwMMN4oKpa8VOo",
"tvjh1g0Xygp6DMsH8n5hX41jgbNh3VE61MsnWHYkfZw",
"d_8_FhktHjeMh91gB9bBKGHgMJFC6auZqHThhlHL9Hw",
"P1RAwTjkXvZUQLjHyi06Zm1YP1KeaZMJdjaNN1ZeOeE",
"DQV3KQ2U9DLoro0jMQUoIr-PQMKBpKIEpOqXzF1A498",
"6IcCytBh4PkKOC3YE2Smw4IGdAyTD7XTtsmgtxF-kdY",
"1EotnhALO_Xf7d5mlOVefECdfpbgPG_Dik0TeOcQauQ",
"92hxCAU523jWUFRXMhM-p3w66P3PxYfY_tZm9pPeShU",
"KwvOafjkobVhF8Iz63lMyNooS-hjaYgmHdZrqVVVHWw",
"UfHD68dH4O9ld-NS22Se6Bls98P9aPZhFq4hA_H7-74",
"c6NrKs_BpAX4OdRFKAVSDeUaQTjSpNF8dCxjcLT6AfE",
"mwA5L1GvWFvnwIYL-U1kP0RDsZ61ElOKAH7wnbsTs4M"
]

In [16]:
# Hàm gọi api here để lấy thông tin vị trí từ địa chỉ chi tiết
import requests

In [17]:

def get_location(address, api_key):
    #api_key = "mwA5L1GvWFvnwIYL-U1kP0RDsZ61ElOKAH7wnbsTs4M"

    url = f'https://geocode.search.hereapi.com/v1/geocode?q={address}&apiKey={api_key}'
    
    try:
        # Gửi yêu cầu API
        response = requests.get(url)
        
        # Nếu API key đã hết hạn, thử API key tiếp theo trong danh sách

        if response.status_code == 429:
            return {"lat": "", "lng": "", "House_number": "", "Street": "", "Ward": "", "District": "", "City": "", "progress": "False" }
        else: 
            data = response.json()    
            # Xử lý kết quả và trả về
            if data == {'items': []}: 
                list_address = {"lat": "", "lng": "", "House_number": "", "Street": "", "Ward": "", "District": "", "City": "", "progress": "True" }
            else: 
                result = data['items'][0]
                position = result['position']
                lat = position.get('lat', '')
                long = position.get('lng', '')
                address_info = result['address']
                city = address_info.get('county', '')
                district = address_info.get('city', '')
                ward = address_info.get('district', '')
                street = address_info.get('street', '')
                houseNumber = address_info.get('houseNumber', '')
                list_address = {"lat": lat, "lng": long, "House_number": houseNumber, "Street": street, "Ward": ward, "District": district, "City": city, "progress": "True" }
    except:
           # Nếu thử xảy ra lỗi trả ra tập rỗng và progess là False đánh dấu những case chưa xử lí
        list_address = {"lat": "", "lng": "", "House_number": "", "Street": "", "Ward": "", "District": "", "City": "", "progress": "False" }
           
    return list_address

# Hàm xử lí dữ liệu đầu vào là dataframe
def out_put(df, api_key):
    house_numbers = []
    streets = []
    wards = []
    districts = []
    cities = []
    progresses = []
    
    for address in df["address"]:
        address_info = get_location(address,api_key)
        house_numbers.append(address_info["House_number"])
        streets.append(address_info["Street"])
        wards.append(address_info["Ward"])
        districts.append(address_info["District"])
        cities.append(address_info["City"])
        progresses.append(address_info["progress"])
    
    df["House_number"] = house_numbers
    df["Street"] = streets
    df["Ward"] = wards
    df["District"] = districts
    df["City"] = cities
    df["progress"] = progresses

    return df

In [18]:
df_test = df

In [19]:
# Calculate the number of groups
num_groups = len(df_test) // 1000

# Calculate the number of remaining rows
remaining_rows = len(df_test) % 1000

# Create a list to store the new dataframes
dfs = []

# Split the dataframe into groups of two rows each
for i in range(num_groups):
    start_index = i * 1000
    end_index = start_index + 1000
    df_group = df_test[start_index:end_index].copy()
    dfs.append(df_group)

# Add the remaining rows to the last dataframe
if remaining_rows > 0:
    last_df = df_test[-remaining_rows:].copy()
    dfs.append(last_df)

In [20]:
# tiến hành chạy multi thread
import threading
import time

final = []
def out_put_with_time(df, api_key):
    start_time = time.time()
    out_put(df, api_key)
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Time: {execution_time/60} phút")

for df , api_key in zip(dfs, list_api_keys):
    thread = threading.Thread(target=out_put_with_time, args=(df,api_key))
    thread.start()
    thread.join()
    final.append(df)

print("Done")

Time: 3.992691437403361 phút
Time: 3.8345983068148297 phút
Done


In [21]:
merged_df = pd.concat(final, ignore_index=True)
merged_df.info()
# merged_df.head(4)
#check case false


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1955 entries, 0 to 1954
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Title         1955 non-null   object 
 1   Price         1955 non-null   object 
 2   Area          1955 non-null   object 
 3   address       1955 non-null   object 
 4   area_value    1955 non-null   float64
 5   price_per_m2  1955 non-null   float64
 6   House_number  1955 non-null   object 
 7   Street        1955 non-null   object 
 8   Ward          1955 non-null   object 
 9   District      1955 non-null   object 
 10  City          1955 non-null   object 
 11  progress      1955 non-null   object 
dtypes: float64(2), object(10)
memory usage: 183.4+ KB


In [22]:
merged_df1 = merged_df[merged_df["progress"] != "False"]
merged_df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1955 entries, 0 to 1954
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Title         1955 non-null   object 
 1   Price         1955 non-null   object 
 2   Area          1955 non-null   object 
 3   address       1955 non-null   object 
 4   area_value    1955 non-null   float64
 5   price_per_m2  1955 non-null   float64
 6   House_number  1955 non-null   object 
 7   Street        1955 non-null   object 
 8   Ward          1955 non-null   object 
 9   District      1955 non-null   object 
 10  City          1955 non-null   object 
 11  progress      1955 non-null   object 
dtypes: float64(2), object(10)
memory usage: 198.6+ KB


In [23]:
df2 = merged_df[merged_df["progress"] == "False"]
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Title         0 non-null      object 
 1   Price         0 non-null      object 
 2   Area          0 non-null      object 
 3   address       0 non-null      object 
 4   area_value    0 non-null      float64
 5   price_per_m2  0 non-null      float64
 6   House_number  0 non-null      object 
 7   Street        0 non-null      object 
 8   Ward          0 non-null      object 
 9   District      0 non-null      object 
 10  City          0 non-null      object 
 11  progress      0 non-null      object 
dtypes: float64(2), object(10)
memory usage: 0.0+ bytes


In [24]:
def get_location(address, api_key):
    #api_key = "mwA5L1GvWFvnwIYL-U1kP0RDsZ61ElOKAH7wnbsTs4M"

    url = f'https://geocode.search.hereapi.com/v1/geocode?q={address}&apiKey={api_key}'
    
    try:
        # Gửi yêu cầu API
        response = requests.get(url)
        
        # Nếu API key đã hết hạn, thử API key tiếp theo trong danh sách

        if response.status_code == 429:
            return {"lat": "", "lng": "", "House_number": "", "Street": "", "Ward": "", "District": "", "City": "", "progress": "False" }
        else: 
            data = response.json()    
            # Xử lý kết quả và trả về
            if data == {'items': []}: 
                list_address = {"lat": "", "lng": "", "House_number": "", "Street": "", "Ward": "", "District": "", "City": "", "progress": "True" }
            else: 
                result = data['items'][0]
                position = result['position']
                lat = position.get('lat', '')
                long = position.get('lng', '')
                address_info = result['address']
                city = address_info.get('county', '')
                district = address_info.get('city', '')
                ward = address_info.get('district', '')
                street = address_info.get('street', '')
                houseNumber = address_info.get('houseNumber', '')
                list_address = {"lat": lat, "lng": long, "House_number": houseNumber, "Street": street, "Ward": ward, "District": district, "City": city, "progress": "True" }
    except:
           # Nếu thử xảy ra lỗi trả ra tập rỗng và progess là False đánh dấu những case chưa xử lí
        list_address = {"lat": "", "lng": "", "House_number": "", "Street": "", "Ward": "", "District": "", "City": "", "progress": "False" }
           
    return list_address

# Hàm xử lí dữ liệu đầu vào là dataframe
def out_put(df2, api_key):
    house_numbers = []
    streets = []
    wards = []
    districts = []
    cities = []
    progresses = []
    
    for address in df2["address"]:
        address_info = get_location(address,api_key)
        house_numbers.append(address_info["House_number"])
        streets.append(address_info["Street"])
        wards.append(address_info["Ward"])
        districts.append(address_info["District"])
        cities.append(address_info["City"])
        progresses.append(address_info["progress"])
    
    df2["House_number"] = house_numbers
    df2["Street"] = streets
    df2["Ward"] = wards
    df2["District"] = districts
    df2["City"] = cities
    df2["progress"] = progresses

    return df2

In [25]:
df_test2 = df2

In [26]:
# Calculate the number of groups
num_groups = len(df_test2) // 1000

# Calculate the number of remaining rows
remaining_rows = len(df_test2) % 1000

# Create a list to store the new dataframes
dfs2 = []

# Split the dataframe into groups of two rows each
for i in range(num_groups):
    start_index = i * 1000
    end_index = start_index + 1000
    df_group2 = df_test2[start_index:end_index].copy()
    dfs2.append(df_group2)

# Add the remaining rows to the last dataframe
if remaining_rows > 0:
    last_df2 = df_test2[-remaining_rows:].copy()
    dfs2.append(last_df2)

In [27]:
# tiến hành chạy multi thread
import threading
import time

final2 = []
def out_put_with_time(df2, api_key):
    start_time = time.time()
    out_put(df2, api_key)
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Time: {execution_time/60} phút")

for df2 , api_key in zip(dfs2, list_api_keys):
    thread = threading.Thread(target=out_put_with_time, args=(df2,api_key))
    thread.start()
    thread.join()
    final2.append(df2)

print("Done")

Done


In [28]:
merged_df2 = pd.concat(final2, ignore_index=True)
merged_df2.info()


ValueError: No objects to concatenate

In [29]:
merged_df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1955 entries, 0 to 1954
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Title         1955 non-null   object 
 1   Price         1955 non-null   object 
 2   Area          1955 non-null   object 
 3   address       1955 non-null   object 
 4   area_value    1955 non-null   float64
 5   price_per_m2  1955 non-null   float64
 6   House_number  1955 non-null   object 
 7   Street        1955 non-null   object 
 8   Ward          1955 non-null   object 
 9   District      1955 non-null   object 
 10  City          1955 non-null   object 
 11  progress      1955 non-null   object 
dtypes: float64(2), object(10)
memory usage: 198.6+ KB


In [ ]:
merge_final = pd.concat([merged_df1,merged_df2], ignore_index=True)
merge_final.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1955 entries, 0 to 1954
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Title         1955 non-null   object 
 1   Price         1955 non-null   object 
 2   Area          1955 non-null   object 
 3   address       1955 non-null   object 
 4   area_value    1955 non-null   float64
 5   price_per_m2  1955 non-null   float64
 6   House_number  1955 non-null   object 
 7   Street        1955 non-null   object 
 8   Ward          1955 non-null   object 
 9   District      1955 non-null   object 
 10  City          1955 non-null   object 
 11  progress      1955 non-null   object 
dtypes: float64(2), object(10)
memory usage: 183.4+ KB


In [31]:
# merged_df['City'] = np.where(merged_df['City'].str.contains('[a-zA-Z]'),merged_df['City'],merged_df['city'])
# merged_df['District'] = np.where(merged_df['District'].str.contains('[a-zA-Z]'),merged_df['District'],merged_df['district'])
merged_df1 = merged_df1.rename(columns={'price_per_m2':'price_per_m2_final'})
merged_df1['price_value_trieu'] = merged_df1['price_per_m2_final']*merged_df1['area_value']/1000000


In [32]:
df_final_nhaphonet = merged_df1[['City','District','Ward','Street','address','area_value','price_value_trieu','price_per_m2_final']]
df_final_nhaphonet.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1955 entries, 0 to 1954
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   City                1955 non-null   object 
 1   District            1955 non-null   object 
 2   Ward                1955 non-null   object 
 3   Street              1955 non-null   object 
 4   address             1955 non-null   object 
 5   area_value          1955 non-null   float64
 6   price_value_trieu   1955 non-null   float64
 7   price_per_m2_final  1955 non-null   float64
dtypes: float64(3), object(5)
memory usage: 137.5+ KB


In [ ]:
# df_final_nhaphonet = pd.read_csv(r"D:\OneDrive\KiotViet\Python_for_work\KFinance\CSV_crawl_data\final_bds_nhaphonet.csv")


In [ ]:
df_final_nhaphonet['price_per_m2_final'] = df_final_nhaphonet['price_per_m2']
df_final_nhaphonet = df_final_nhaphonet[['City','District','Ward','Street','address','area_value','price_per_m2_final']]


In [33]:
df_final_nhaphonet.to_csv(r"D:\OneDrive\KiotViet\Python_for_work\KFinance\CSV_crawl_data\final_bds_nhaphonet.csv", index=False, mode = 'w', encoding="utf-8-sig")